# Acquiring pointing data for AuxTel

The first step in this notebook is to load all the required libraries. 

In [1]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS

In [2]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5

Before proceeding, add a logger and set logging level to `DEBUG` so we can get status message from the various tasks.

In [3]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

Next we initialize two high level classes to command the ATTCS and LATISS.

In [4]:
attcs = ATTCS()

Using selector: EpollSelector


In [5]:
await attcs.start_task

Read historical data in 0.00 sec
Read historical data in 0.23 sec
Read historical data in 0.78 sec
Read historical data in 1.27 sec
Read historical data in 2.05 sec
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 55 messages
Read historical data in 2.93 sec
Read historical data in 4.15 sec
RemoteTelemetry(ATDome, 0, position) falling behind; read 24 messages


[None, None, None, None, None, None, None]

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 15 messages


In [6]:
await asyncio.sleep(5.)

RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 18 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 38 messages


In [349]:
await salobj.set_summary_state(attcs.atmcs, salobj.State.ENABLED)

RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, skyEnvironment) falling behind; read 36 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [350]:
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

# Bet Ori

In [ ]:
await attcs.slew_icrs(ra="05 14 32.27210", dec="-08 12 05.8981", rot_pa=-90, target_name="Bet Ori", slew_timeout=120)

# alf CMa

In [13]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="06 45 08.91728", dec="-16 42 58.0171", rot_pa=-90., target_name="alf CMa", slew_timeout=240)

RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 75 messages
RemoteTelemetry(ATPtg, 0, skyEnvironment) falling behind; read 22 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
Flushing events
Scheduling check coroutines
Sending command
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 77 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 

KeyboardInterrupt: 

In [42]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=-0.165*60.*60.,
                                             off2=0.385*60.*60., 
                                      num=0)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 69 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 14 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 14 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 72 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 14 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 15 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 14 messages


In [43]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 62 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 12 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 12 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 13 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 13 messages


# alf Car

In [59]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="06 23 57.10988", dec="-52 41 44.3810", rot_pa=-90., target_name="alf Car", slew_timeout=240)


RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 21 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 21 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 21 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 21 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 15 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 15 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 21 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 21 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, detailedState) fa

In [46]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=0.*60.*60.,
                                             off2=0.*60.*60., 
                                      num=0)

RemoteEvent(ATAOS, 0, detailedState) falling behind; read 71 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 51 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 36 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 36 messages
RemoteTelemetry(ATPtg, 0, skyEnvironment) falling behind; read 11 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 51 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 52 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 52 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 52 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 52 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 11 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 53 messages


In [60]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=0.200*60.*60.,
                                             off2=-0.429*60.*60., 
                                      num=0)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 35 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 33 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 35 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 35 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 10 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 36 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 12 messages


In [58]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 82 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 17 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 17 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 17 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 41 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 41 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 12 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 17 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 18 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 18 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 88 messages


In [73]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="06 23 57.10988", dec="-52 41 44.3810", rot_pa=-45., target_name="alf Car", slew_timeout=240)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 98 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 20 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 97 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 28 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 14 messages
Flushing events
Scheduling check coroutines
Sending command
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 20 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 21 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 20 messages
RemoteEvent(ATDome, 0, shutterInPosition) f

In [71]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=0.172*60.*60.,
                                             off2=-0.435*60.*60., 
                                      num=0)

RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 11 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 50 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 51 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 11 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 24 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 52 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 10 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 24 messages


In [72]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 61 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 13 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 12 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 18 messages


In [ ]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="06 23 57.10988", dec="-52 41 44.3810", rot_pa=45., target_name="alf Car", slew_timeout=240)

In [83]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=0.150*60.*60.,
                                             off2=-0.438*60.*60., 
                                      num=0)

RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 19 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 93 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 19 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 13 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 19 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 19 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 20 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 36 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 20 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 27 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 97 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 20 messages
RemoteEvent(ATDome, 0, h

In [93]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=0.125*60.*60.,
                                             off2=-0.442*60.*60., 
                                      num=0)

RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 69 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 13 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 13 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 69 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 14 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 72 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages


In [94]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 88 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 86 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 26 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 89 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 18 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 18 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 18 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 19 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 18 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 18 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 19 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 18 messages
RemoteTelemetry(ATMCS, 

# bet Car

In [ ]:
attcs.check.atdome = True
await attcs.slew_icrs(ra="09 13 11.97746", dec="-69 43 01.9473", rot_pa=0., target_name="bet Car", slew_timeout=240)

RemoteEvent(ATMCS, 0, target) falling behind; read 79 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 21 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 20 messages
Flushing events
Scheduling check coroutines
Sending command
Got False
ATDome not in position
Got True
ATDome in position.
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 22 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 22 messages
process as completed...
[Telescope] delta Alt = -000.001 | delta Az= +000.007 [Dome] delta Az = +003.980
Got False
Telescope not in position
[Telescope] delta Alt = -020.129 | delta Az= -028.654 [Dome] delta Az = -024.451
[Telescope] delta Alt = -019.662 | delta Az= -028.410 [Dome] delta Az = -023.308
[Telescope] delta Alt = -018.059 | delta Az= -027.498 [Dome] delta Az = -021.235
[Telescope] delta Alt = -015.225 | delta Az= -025.708 [Dome

In [151]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=1.6*60.*60.,
                                            off2=-0.03*60.*60., 
                                            num=0)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 33 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 33 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 34 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages


In [152]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 24 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 25 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 17 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 24 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 25 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 25 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 61 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 17 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 25 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 25 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEve

In [153]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="09 13 11.97746", dec="-69 43 01.9473", rot_pa=-45., target_name="bet Car", slew_timeout=240)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 21 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 29 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 21 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 20 messages
Flushing events
Scheduling check coroutines
Sending command
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 15 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 21 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 21 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 34 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind

In [164]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=1.60*60.*60.,
                                            off2=-0.044*60.*60., 
                                            num=0)

RemoteTelemetry(ATDome, 0, position) falling behind; read 36 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 38 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 39 messages


In [165]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 11 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 73 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 15 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 15 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 15 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 75 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 27 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 15 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 10 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 15 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 15 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 27 messages
RemoteEven

In [166]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="09 13 11.97746", dec="-69 43 01.9473", rot_pa=-90., target_name="bet Car", slew_timeout=240)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 70 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 14 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 69 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 14 messages
Flushing events
Scheduling check coroutines
Sending command
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 10 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 14 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 10 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 31 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) fa

In [178]:
await attcs.atptg.cmd_offsetRADec.set_start(off1=1.595*60.*60.,
                                            off2=-0.058*60.*60., 
                                            num=0)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 65 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 64 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 13 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 13 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 13 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 23 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 23 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 13 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 13 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages


In [179]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 12 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 80 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 12 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 17 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 34 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 17 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 23 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 35 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 17 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 17 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 17 messages
RemoteTelemetry(ATMCS, 0, trajectory) fall

# gam02 Vel

In [181]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="08 09 31.95013", dec="-47 20 11.7108", rot_pa=0., target_name="gam02 Vel", slew_timeout=240)


RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, skyEnvironment) falling behind; read 36 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) fa

In [202]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.3625*60.*60., 
                                     az=0.1515*60.*60., 
                                     num=0)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 56 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 39 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 39 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 39 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 39 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 98 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 40 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 

In [203]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 10 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 10 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 15 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 15 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 10 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 15 messages


In [204]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="08 09 31.95013", dec="-47 20 11.7108", rot_pa=90., target_name="gam02 Vel", slew_timeout=240)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 62 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 64 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 13 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 64 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 13 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 22 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 13 messages
RemoteEvent(ATDome, 0, ma

In [237]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.3700*60.*60., 
                                     az=0.145*60.*60., 
                                     num=0)

RemoteTelemetry(ATDome, 0, position) falling behind; read 39 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 22 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 42 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 11 messages


In [238]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 14 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 14 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 14 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 32 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 15 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 15 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 14 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 32 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 15 messages
Remot

# alf CMi

In [239]:
attcs.check.atdome = True
await attcs.slew_icrs(ra="07 39 18.11950", dec="05 13 29.9552", rot_pa=90., target_name="alf CMi", slew_timeout=240)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 31 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 31 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 31 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 31 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 31 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 31 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 73 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 31 messages
RemoteEvent(ATAOS, 0, m1Corr

In [254]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.39*60.*60., 
                                     az=0.430*60.*60., 
                                     num=0)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 57 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 12 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 16 messages


In [255]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 82 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 12 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 86 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 87 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages


In [256]:
attcs.check.atdome = True
await attcs.slew_icrs(ra="07 39 18.11950", dec="05 13 29.9552", rot_pa=-90., target_name="alf CMi", slew_timeout=240)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 26 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 26 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 26 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 26 messages
Flushing events
Scheduling check coroutines
Sending command
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 26 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 26 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 18 messages
Got False
ATDome not in position
Got True
ATDome in position.
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 57 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 26 messages
RemoteEvent

In [261]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.385*60.*60., 
                                     az=0.430*60.*60., 
                                           num=0)

RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 21 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 20 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 21 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 21 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 28 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 21 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 m

In [262]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 68 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 15 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 15 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 19 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 35 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 14 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 10 messages
RemoteEvent(ATPneumatics

In [278]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.3915*60.*60., 
                                     az=0.435*60.*60., 
                                           num=0)

RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 12 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 12 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 12 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 12 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 12 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 12 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 12 messages


In [279]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 67 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 14 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 10 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 14 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 38 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 72 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 10 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 73 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 14 messages
RemoteTelem

# mu Vel

In [280]:
attcs.check.atdome = True
await attcs.slew_icrs(ra="10 46 46.178770", dec="-49 25 12.9244", rot_pa=0., target_name="mu Vel", slew_timeout=240)


RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 66 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 66 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 46 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 66 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 65 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 66 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 66 messages
Flushing events
Scheduling check coroutines
Sending command
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 67 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 46 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 65 messages
RemoteT

In [295]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.348*60.*60., 
                                           az=0.337*60.*60., 
                                           num=0)

RemoteTelemetry(ATDome, 0, position) falling behind; read 54 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 11 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 11 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 11 messages


In [296]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 77 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 80 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 35 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 81 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 16 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 35 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 11 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 17 messages


# alf Crux

In [296]:
attcs.check.atdome = True
await attcs.slew_icrs(ra="12 26 35.896", dec="-63 05 56.73", rot_pa=90., target_name="alf Crux", slew_timeout=240)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATPtg, 0, mountDataWarning) falling behind; read 48 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, skyEnvironment) falling behind; read 38 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positio

KeyboardInterrupt: 

In [314]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.370*60.*60., 
                                           az=0.475*60.*60., 
                                           num=0)

RemoteTelemetry(ATDome, 0, position) falling behind; read 34 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 37 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 10 messages


In [315]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 18 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 12 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 89 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 18 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 18 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 38 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 19 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 19 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 19 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 18 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 19 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 19 messages


# Procyon

In [316]:
attcs.check.atdome = True
await attcs.slew_icrs(ra="7:39:18.12", dec="+5:13:0.0", rot_pa=-90., target_name="Procyon", slew_timeout=240)


RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 100 messages
RemoteT

In [346]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.410*60.*60., 
                                           az=0.445*60.*60., 
                                           num=0)

RemoteTelemetry(ATDome, 0, position) falling behind; read 32 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 33 messages


In [347]:
await attcs.atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await attcs.atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 72 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 15 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 15 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 10 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 10 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 35 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 76 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 15 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 35 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 15 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 15 messages
RemoteEvent(ATPn

# Sirius

In [348]:
attcs.check.atdome = True
await attcs.slew_icrs(ra="6 45 08.92", dec="-16 42 58.0", rot_pa=0., target_name="Sirius", slew_timeout=240)



RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 70 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 68 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 14 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 10 messages
Flushing events
Scheduling check coroutines
Sending command
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 14 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 10 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
Got False
ATDome not in position
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 71 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 30 messages
RemoteTelemetry(ATMCS, 0, 

RuntimeError: atmcs state is <State.FAULT: 3>

In [354]:
await attcs.atpneumatics.cmd_openM1Cover.start()

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 32 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
Task exception was never retrieved
future: <Task finished coro=<ATTCS.wait_for_inposition() done, defined at /home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py:760> exception=TimeoutError()>
Traceback (most recent call last):
  File "/home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py", line 782, in wait_for_inposition
    status = await asyncio.gather(*wait_tasks)
  File "/home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py", line 845, in wait_for_atdome_inposition
    timeout=timeout)
  File "/home/saluser/repos/ts_salobj/python/lsst/ts/salobj/topics/read_topic.py", line 391, in next
    return await self._next(timeout=timeout)
  File "/home/saluser/repos/ts_salobj/pyth

In [355]:
await asyncio.sleep(5)
await attcs.shutdown()

RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 12 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 13 messages
Disabling ATAOS corrections
Close telescope cover.
Disable ATDomeTrajectory


RuntimeError: Error on cmd=cmd_disable, initial_state=2

In [356]:
await attcs.atdome.cmd_closeShutter.start()

RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 32 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 32 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 32 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 32 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 64 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 31 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 64 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 32 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 32 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirP

In [358]:
await attcs.atdome.cmd_moveAzimuth.set_start(azimuth=200)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, skyEnvironment) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 65 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 65 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 66 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 66 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 65 messages
RemoteEvent(ATDome, 0, shutterInPosition) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 66 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 66 messages
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 100 messag

In [361]:
await attcs.atptg.cmd_azElTarget.set_start(azDegs=0., elDegs=78.)

RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 13 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 66 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 13 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 14 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 14 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 13 messages
R

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=169365441, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [ ]:
await 

In [362]:
await asyncio.sleep(5)
await asyncio.gather(salobj.set_summary_state(attcs.atmcs, salobj.State.STANDBY, timeout=120),
                     salobj.set_summary_state(attcs.atptg, salobj.State.STANDBY),
                     salobj.set_summary_state(attcs.ataos, salobj.State.STANDBY, settingsToApply="measured_20190908.yaml"),
                     salobj.set_summary_state(attcs.atpneumatics, salobj.State.STANDBY),
                     salobj.set_summary_state(attcs.athexapod, salobj.State.STANDBY, settingsToApply="Default1"),
                     salobj.set_summary_state(attcs.atdome, salobj.State.STANDBY, settingsToApply="test.yaml"),
                     salobj.set_summary_state(attcs.atdometrajectory, salobj.State.STANDBY))

RuntimeError: Error on cmd=cmd_disable, initial_state=2